In [1]:
from datetime import datetime, date
import pandas as pd
import numpy as np

import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeCV
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, r2_score

from catboost import CatBoostClassifier, Pool, CatBoostRegressor
import plotly

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Необходимые данные для выборок. Список необходимых фич для команд.

In [2]:
with open('cols_order.txt') as f:
    correct_order = f.read().splitlines() 

with open('for_ht.txt') as f:
    columns_for_ht = f.read().splitlines() 
    
with open('for_at.txt') as f:
    columns_for_at = f.read().splitlines() 

In [5]:
FILE_NAME = 'italy_matches.csv'

Данные собирались и агрегировались мною (python + selenium), здесь будет только результат. Но если необходимо, можно опубликовать и парсер. 

Данные по матчам собраны за последние 8 сезонов в рамках одной футбольной лиги (Серия А).

In [6]:
data = pd.read_csv(FILE_NAME, index_col=[0], parse_dates=['match_date'])
data = data.reset_index()
data = data.drop('index', axis=1)

data = data.drop_duplicates(subset='match_id')

# Вычислим победителя матча. 
# 0 - ничья, 1 - победа домашней команды, 2 - победа гостевой команды
data['goals_diff'] = data['ht_goals'] - data['at_goals']
data['winner'] = data['goals_diff'].apply(lambda x: 0 if x == 0 else (1 if x > 0 else 2))
data = data.drop(['goals_diff'], axis=1)

data = data[correct_order]

In [7]:
# Для вычисления средних показателей по сезону (Турнирное положение)
# Определяем team_id - домашняя или гостевая команда, в зависимости от этого выбираем нужные фичи
# Еще вернем общее количество сыгранных матчей

def get_result_before_match(team_id, match_list, cols_names):
    ht_matches = match_list[match_list.ht_id == team_id]
    at_matches = match_list[match_list.at_id == team_id]

    ht_matches = ht_matches[columns_for_ht]
    ht_matches.columns = cols_names

    at_matches = at_matches[columns_for_at]
    at_matches.columns = cols_names

    mean_stats = pd.concat([ht_matches, at_matches]) \
        .add_prefix('mean_') \
        .mean() \
        .to_frame().T

    mean_stats['matches_played'] = len(match_list)
    
    return mean_stats

In [8]:
# Функция для вычисления основных турнирных показателей: очки, количество забитых и пропущенных голов

def get_stats(team_id, match_list, prefix, last_5=False):
    points = 0
    ht_matches = match_list[match_list.ht_id == team_id]
    at_matches = match_list[match_list.at_id == team_id]    
    
    goals_scored = ht_matches.ht_goals.sum() + at_matches.at_goals.sum()
    goals_conc = ht_matches.at_goals.sum() + at_matches.ht_goals.sum()
    
    away_goals = at_matches.at_goals - at_matches.ht_goals
    home_goals = ht_matches.ht_goals - ht_matches.at_goals
    
    goals_diff = pd.concat([away_goals, home_goals]).tolist()
    for diff in goals_diff:
        if diff > 0:
            points += 3
        elif diff == 0:
            points += 1 
    
    df = pd.DataFrame({'points': points, 
               'goals_scored': goals_scored, 
               'goals_conc': goals_conc}, index=[0])
    
    prefix = f'{prefix}l5_' if last_5 else prefix
    
    df = df.add_prefix(prefix)
    
    return df 

Обрабатываем все данные, используя ранее написанные функции.
Это non-pandas-way, но реализовать это без цикла по каждой строке у меня не получилось. Если можете помочь, пишите, буду благодарен.

Необходимо для каждой встречи рассчитать следующие показатели:
    1. Турнирное положение
    2. Средние показатели по каждой фиче в зависимости от места проведения игры (домашняя/выездная)
    3. Для определения формы команды - Турнирное положение за последние 5 матчей
    
В дальнейшей этот список можно будет расширить, используя все те же функции

In [9]:
seasons = np.unique(data.season)
stats = pd.DataFrame()
for season in seasons:
    curr_season = data[data.season == season] \
        .sort_values(by='match_date')
    
    match_info = curr_season.iloc[:, :9]
    
    l5_stats = pd.DataFrame()
    mean_stats = pd.DataFrame()
    standing = pd.DataFrame()
    for i in range(len(curr_season)):
        curr_match_id = curr_season.iloc[i, 1]
        curr_date = curr_season.iloc[i, 2]
        ht_id = curr_season.iloc[i, 4]
        at_id = curr_season.iloc[i, 6]
        
        matches_b4_today = curr_season[curr_season.match_date < curr_date]        
        ht_matches_b4_today = matches_b4_today[(matches_b4_today.ht_id == ht_id) | (matches_b4_today.at_id == ht_id)]
        at_matches_b4_today = matches_b4_today[(matches_b4_today.ht_id == at_id) | (matches_b4_today.at_id == at_id)]

        ht_mean = get_result_before_match(ht_id, ht_matches_b4_today, columns_for_ht)
        at_mean = get_result_before_match(at_id, at_matches_b4_today, columns_for_at)
        ht_mean.columns = ht_mean.columns.str.replace('matches_played', 'ht_matches_played')
        at_mean.columns = at_mean.columns.str.replace('matches_played', 'at_matches_played')
        
        # l5 = 5 последних матчей
        ht_l5 = get_stats(ht_id, ht_matches_b4_today.iloc[-5:, :], 'ht_', True)
        at_l5 = get_stats(at_id, at_matches_b4_today.iloc[-5:, :], 'at_', True)
        l5_df = pd.concat([ht_l5, at_l5], axis=1)
        l5_df['match_id'] = curr_match_id
        
        # Турнирное положение на данный момент
        ht_standing = get_stats(ht_id, ht_matches_b4_today, 'ht_')
        at_standing = get_stats(at_id, at_matches_b4_today, 'at_')
        standing_df = pd.concat([ht_standing, at_standing], axis=1)
        standing_df['match_id'] = curr_match_id          

        mean_df = pd.concat([ht_mean, at_mean], axis=1)
        mean_df['match_id'] = curr_match_id
        
        l5_stats = l5_stats.append(l5_df)
        mean_stats = mean_stats.append(mean_df)
        standing = standing.append(standing_df)
        
    full_season_stats = match_info \
        .merge(mean_stats, on='match_id') \
        .merge(l5_stats, on='match_id') \
        .merge(standing, on='match_id')
        
    stats = stats.append(full_season_stats)
    print(f'{season} done')
    
stats = stats.reset_index(drop=True)

2012/2013 done
2013/2014 done
2014/2015 done
2015/2016 done
2016/2017 done
2017/2018 done
2018/2019 done
2019/2020 done


In [10]:
stats.head()

,season,match_id,match_date,ht_name,ht_id,at_name,at_id,ht_goals,at_goals,mean_ht_goals,...,ht_l5_goals_conc,at_l5_points,at_l5_goals_scored,at_l5_goals_conc,ht_points,ht_goals_scored,ht_goals_conc,at_points,at_goals_scored,at_goals_conc
0,2012/2013,651496,2012-08-25,Ювентус,87,Парма,82,2.0,0.0,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,2012/2013,651494,2012-08-25,Фиорентина,73,Удинезе,86,2.0,1.0,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,2012/2013,651493,2012-08-26,Кьево,267,Болонья,71,2.0,0.0,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,2012/2013,651495,2012-08-26,Дженоа,278,Кальяри,78,2.0,0.0,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,2012/2013,651501,2012-08-26,Сиена,773,Торино,72,0.0,0.0,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [11]:
# Некоторые фичи необходимо преобразовать, упущение при сборе данных)
stats.mean_ht_drible_suc = stats.mean_ht_drible_suc / 100
stats.mean_at_drible_suc = stats.mean_at_drible_suc / 100

stats.mean_ht_otbor_perc = stats.mean_ht_otbor_perc / 100
stats.mean_at_otbor_perc = stats.mean_at_otbor_perc / 100

stats.mean_ht_possession = stats.mean_ht_possession / 100
stats.mean_at_possession = stats.mean_at_possession / 100

Удалим первые 5 матчей каждого сезона, т.к. они малоинформативны

In [12]:
stats = stats[(stats.ht_matches_played >= 5) | (stats.at_matches_played >= 5)]

Попробуем решить задачу классификации победителя матча разными методами.

In [15]:
stats['goals_diff'] = stats['ht_goals'] - stats['at_goals']
stats['winner'] = stats['goals_diff'].apply(lambda x: 0 if x == 0 else (1 if x > 0 else 2))
stats = stats.drop(['goals_diff'], axis=1)

In [16]:
np.unique(stats.winner, return_counts=True)

(array([0, 1, 2], dtype=int64), array([ 669, 1178,  796], dtype=int64))

Классы несбалансированны. Для баланса воспользуемся SMOTE

In [17]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [18]:
x = stats.loc[:, 'mean_ht_goals':'at_goals_conc']
y = stats.winner

In [19]:
x_resampled, y_resampled = SMOTE().fit_resample(x, y)
np.unique(y_resampled, return_counts=True)

x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2)

In [20]:
# x_resampled, y_resampled = RandomUnderSampler().fit_resample(x, y)
# np.unique(y_resampled, return_counts=True)

In [21]:
def get_metrics(y_test, y_pred):
    pr, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Prec = {round(pr, 3)} \n Rec = {round(rec, 3)} \n  F1 = {round(f1, 3)}')

In [22]:
kn = KNeighborsClassifier(n_neighbors=8)
kn.fit(x_train, y_train)

y_pred = kn.predict(x_test)
get_metrics(y_test, y_pred)

Prec = 0.501 
 Rec = 0.495 
  F1 = 0.494


In [23]:
svc = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

get_metrics(y_test, y_pred)

Prec = 0.567 
 Rec = 0.566 
  F1 = 0.565


In [24]:
gbc = GradientBoostingClassifier(learning_rate=0.05, n_estimators=300, min_samples_split=5,
                                 min_samples_leaf=3, max_depth=5)
gbc.fit(x_train, y_train)
y_pred = gbc.predict(x_test)

get_metrics(y_test, y_pred)

Prec = 0.608 
 Rec = 0.607 
  F1 = 0.607


Параметры для методов подбирались с помощью RandomizedSearchCV. Наиболее продуктивная модель GradientBoostingClassifier.

Немного поменяем задачу. Сведем задачу к предсказанию количества голов команд. Ну а далее, кто больше забил - тот и победил. Данный способ напрямую не позволяет получить ничейный результат. Поэтому введем ограничения: если разница голов команд менее, чем 0.15 - признаем ничью.
<br><br>
Для этого выберем фичи, которые напрямую могут влиять на эффективность нападения команд.
Так же рассчитаем среднюю эффективность ударов и сумму атак в целом.

In [25]:
def get_data_ht(data):
    for_ht_regr = data[['mean_ht_wc_rating_avg', 'mean_ht_key_passes', 'mean_ht_corner', 
                        'ht_l5_points', 'ht_l5_goals_scored', 'mean_at_wc_rating_avg', 
                        'at_l5_points', 'at_l5_goals_conc']]
    
    for_ht_regr['attacks'] = data[['mean_ht_att_combination', 'mean_ht_stand_pol', 'mean_ht_contr_att', 
          'mean_ht_penalty', 'mean_ht_own_goal']].sum(axis=1)
    
    for_ht_regr['shots_eff'] = (data[['mean_ht_shot_in_target', 'mean_ht_shot_miss', 
                                  'mean_ht_shot_blocked']].sum(axis=1)) * data.mean_ht_goals
    return for_ht_regr

In [26]:
def get_data_at(data):
    for_at_regr = data[['mean_at_wc_rating_avg', 'mean_at_key_passes', 'mean_at_corner', 
                        'at_l5_points', 'at_l5_goals_scored', 'mean_ht_wc_rating_avg', 
                        'ht_l5_points', 'ht_l5_goals_conc']]
    
    for_at_regr['attacks'] = data[['mean_at_att_combination', 'mean_at_stand_pol', 'mean_at_contr_att', 
          'mean_at_penalty', 'mean_at_own_goal']].sum(axis=1)
    
    for_at_regr['shots_eff'] = (data[['mean_at_shot_in_target', 'mean_at_shot_miss', 
                                  'mean_at_shot_blocked']].sum(axis=1)) * data.mean_at_goals
    return for_at_regr

In [27]:
def get_regr_metrics(y_test, y_pred):
    print(f'MSE  = {mean_squared_error(y_test, y_pred)}')
    print(f'R2   = {r2_score(y_test, y_pred)}')

Таким образом, необходимо 2 модели:
    1. Для домашней команды
    2. Для гостевой команды

In [49]:
stats = stats.dropna()

In [50]:
x = get_data_ht(stats)
y = stats.ht_goals

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=16)

ht_clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1], cv=5).fit(x_train, y_train)
y_pred_ht = ht_clf.predict(x_test)

get_regr_metrics(y_test, y_pred_ht)

MSE  = 1.2176347544173098
R2   = 0.11307048885197102


Аналогично для гостевой команды

In [51]:
x = get_data_at(stats)
y = stats.at_goals

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=16)

at_clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1], cv=5).fit(x_train, y_train)
y_pred_at = at_clf.predict(x_test)

get_regr_metrics(y_test, y_pred_at)

MSE  = 1.2007155737978195
R2   = 0.11499139541084424


Определим победителя и сравним с результатами классификации

In [52]:
prediction = pd.DataFrame({
    'ht_goals_pred': y_pred_ht,
    'at_goals_pred': y_pred_at
})

In [53]:
threshold = 0.2

prediction['goals_diff'] = prediction['ht_goals_pred'] - prediction['at_goals_pred']
prediction['winner'] = prediction['goals_diff'].apply(
    lambda x: 0 if ((x > -threshold) & (x < threshold)) else (1 if x >= threshold else 2)
)
prediction = prediction.drop(['goals_diff'], axis=1)

In [54]:
y_test = stats.loc[x_test.index, :].winner

In [55]:
np.unique(prediction.winner, return_counts=True)

(array([0, 1, 2], dtype=int64), array([135, 281, 113], dtype=int64))

In [56]:
get_metrics(y_test, prediction.winner)

Prec = 0.505 
 Rec = 0.503 
  F1 = 0.499


In [57]:
cbr = CatBoostRegressor(learning_rate=0.01, max_depth=7, iterations=500, silent=True)

In [58]:
x = get_data_ht(stats)
y = stats.ht_goals

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=16)

ht_clf = cbr.fit(x_train, y_train)
y_pred_ht = ht_clf.predict(x_test)

get_regr_metrics(y_test, y_pred_ht)

MSE  = 1.229497734790739
R2   = 0.10442945150871685


In [59]:
pd.DataFrame({
    'feature' : cbr.feature_names_,
    'imp' : cbr.feature_importances_
}).sort_values(by='imp', ascending=False)

,feature,imp
5,mean_at_wc_rating_avg,19.067526
9,shots_eff,15.262149
0,mean_ht_wc_rating_avg,10.828919
8,attacks,8.476900
6,at_l5_points,8.351819
7,at_l5_goals_conc,8.329513
4,ht_l5_goals_scored,8.308885
1,mean_ht_key_passes,7.967409
3,ht_l5_points,6.916374
2,mean_ht_corner,6.490506


In [60]:
x = get_data_at(stats)
y = stats.ht_goals

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=16)

ht_clf = cbr.fit(x_train, y_train)
y_pred_ht = ht_clf.predict(x_test)

get_regr_metrics(y_test, y_pred_ht)

MSE  = 1.2274551200982011
R2   = 0.10591729935811833


In [61]:
pd.DataFrame({
    'feature' : cbr.feature_names_,
    'imp' : cbr.feature_importances_
}).sort_values(by='imp', ascending=False)

,feature,imp
5,mean_ht_wc_rating_avg,22.976563
0,mean_at_wc_rating_avg,13.209719
6,ht_l5_points,11.192796
2,mean_at_corner,8.881780
8,attacks,8.249086
1,mean_at_key_passes,8.074265
3,at_l5_points,8.016128
9,shots_eff,7.215244
4,at_l5_goals_scored,6.265318
7,ht_l5_goals_conc,5.919100


На данном этапе итог работы таков:
<ul>
    <li>Точность метода с использованием регрессии ниже. Метрики для оценки качества регрессии здесь не помогают, ибо при предсказании, например 2:0 и действительном результате 5:0, ошибка будет большая, но по факту - победитель предсказан. Именно поэтому оценка производится приведением результатов к тому же виду, что и результаты классификации. </li><br>
    <li>Если копаться не только в метриках, но еще и смотреть на сами данные, можно заметить, что большинство моделей способны предсказать наиболее очевидные результаты, когда играет заведомо разные по статистике команды, где явно видно преимущество одной над другой. На таких матчах в большинстве случаев коэффициент (в будущем просто кэф) довольно таки низкий. Из этого следует другой вывод. </li><br>
    <li>Метрики для оценивания моделей необходимо привязывать к коэффициентам букмекеров. Мало смысла, когда модель предсказывает победителя с кэфом менее 1.2, а в матчах с кэфом более 2 ошибается. В таком случае лучше один верно предсказанный матч с кэфом 1.5, чем 4 матча с кэфом 1.1. Но и оставлять кэф как фичу вредно, т.к. именно эта фича становится самой важной при предсказании. В таком случае вся модель скатывается к правилу - победит тот, на кого кэф меньше).</li><br>
    <li>Необходимо использовать данные для получения вероятностей вроде:
<p>При преимуществе домашней команды в среднем количестве атак и среднему количеству голов (где не всегда наблюдается корреляция :)) над гостевой командой, гостевая команды выиграла только 14% матчей, при этом в половине из них она не забила.</p>
И отталкиваясь от таких статистических закономерностей строить свою стратегию.</li><br>
    <li>Несмотря на метрики, на данных по текущему сезону, лучше всего отрабатывает подход с использованием регрессии. Его точность по-прежнему чуть выше 0.5, но часто проходят прогнозы с кэфом более 3. Т.е. пока не наблюдается предсказание "очевидных матчей". С точки зрения метрик модель хуже, но с финансовой точки зрения - лучше.</li>
</ul>